# <strong> Example parameterised cradle-to-gate life cycle assessment: Linking BatPaC to Brightway </strong>

Following notebook provides three examples of the linkages between BatPaC to Brightway2 to calculate the cradle-to-gate life cycle emissions of different battery designs.


* [Example 1: Cradle to gate emission of a single battery design](#first-bullet)
* [Example 2: Cradle to gate emission  of several battery design](#second-bullet)
* [Example 3: Cradle to gate emission  of several battery design modular approach](#third-bullet)



<center><img src='../docs/batpac_bw_link.jpg' width=800/></center>

In [1]:
#Import required packages

import brightway2 as bw
import numpy as np
import pandas as pd
import openpyxl
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group, ParameterManager
import sys
import matplotlib.pyplot as plt

#Import the battery design module from a local path
local_path =r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\batt_sust_model" #ADD PATH
sys.path.insert(1, local_path)
from battery_design import battery_design as bd
import battery_emissions as bat_lca

#Local path to BatPaC version 5
path_batpac = r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\battery_sustainability_model\Case study\BatPaC 5.0 - 8March2022.xlsm"#ADD PATH


In [2]:
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Example 1 - Cradle-to-gate of a single battery <a class="anchor" id="first-bullet"></a>




## Solve battery system in BatPaC:
First, the material content of a single battery is obtained. The design of the VW ID.4 pack is used. See also the [example battery design notebook](https://github.com/jbaars2/Batt_Sust_Model/blob/main/example%20notebooks/example_battery_design.ipynb).

In [5]:
#Establish battery system class of specific design:
battery = bd.Battery_system (    
                            vehicle_type='EV', 
                             electrode_pair='NMC622-G (Energy)',                              
                             cells_per_module=24, 
                             modules_per_row=6,
                             rows_of_modules=2, 
                             sep_film_thickness = 18,
                             negative_foil_thickness = 12,
                             positive_foil_thickness = 14,
                             positive_electrode_thickness = 87.3,
                             cells_in_parallel=3, #https://static.nhtsa.gov/odi/tsbs/2021/MC-10186407-0001.pdf
                             modules_in_parallel = 1,  
                             silicon_anode= 0.00, 
                             calculate_fast_charge = 'Yes',
                             max_charging_time = 33,  
                             max_charge_power = 135, 
                             available_energy=94,
                             A_coefficient=135.9999272, #based on EPA data
                             B_coefficient= 3.732390689,#based on EPA data
                             C_coefficient=0.406771986,#based on EPA data
                             vehicle_range_miles =410*0.6214, #EV-database using the 'EVDB Real Range' (converted to miles) https://ev-database.org/car/1273/Volkswagen-ID4-1st
                             motor_power = 150 #Based on adac.de VW ID.4 Pro Performance https://www.adac.de/rund-ums-fahrzeug/autokatalog/marken-modelle/vw/id4/1generation/318521/#technische-daten                 
                             )
#Establish parameter dictionary
parameter_dict = battery.parameter_dictionary()

In [6]:
# #Solve battery design in BatPaC returning nested dictionary with material contents and parameter values
# #Make sure BatPaC is not open or running in the background
result_dict = bd.solve_batpac_battery_system (batpac_path = path_batpac, parameter_dict=parameter_dict, visible=False)

## Import Brightway databases
Brightway databases are imported containing parameterised inventories and default project parameters:


In [2]:
#Brightway2 general setup:
bw.projects.set_current('parameterised_battery_lca')
bw.bw2setup()
#Import ecoinvent 3.7.1:
if 'eidb 3.7' in bw.databases:
    pass
else:
    pathi37 = 'path to ecoinvent'
    eidb37 = bw.SingleOutputEcospold2Importer(pathi37, 'eidb 3.7')
    eidb37.apply_strategies()
    eidb37.statistics()
    
#Battery LCA setup:
# Import all Brightway2 databases (BW2Package format) required for the battery LCA: 
bat_lca.import_db_brightway()

# Import all project parameters and include default values such as process yield and energy consumption
bat_lca.import_project_parameters()

# # Import the BW activity functions CHECK IF NEEDED IF THE BW2PACKAGES ARE UPLOADED!!!!
# df_act_functions = pd.read_excel(r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\batt_sust_model\data\process_formulas.xlsx", sheet_name = 'activity_functions_cut_off')

# bat_lca.import_activity_functions    (df_act_functions)

Biosphere database already present!!! No setup is needed
anode_material  already present
battery_production  already present
cathode_material  already present
cell_container  already present
cell_other  already present
current_collectors  already present
cut_off_production  already present
electrolyte  already present
manufacturing_waste_scrap  already present
module  already present
MyModelName  already present
pack  already present
separator  already present
separators  already present


All battery production activities contain formulas with Brightway project parameters. 
 <br>For example, the activity "mixing cathode materials" in the "battery_production" database contains the following exchanges, formulas and default amounts:

In [9]:
act_cat_mixing = [act for act in bw.Database('battery_production') if act['name']=='mixing cathode materials'][0] 
input = [bw.get_activity(exc['input'])['reference product'] for exc in act_cat_mixing.exchanges()]
formula = [exc['formula'] for exc in act_cat_mixing.exchanges()]
amount = [round(exc['amount'], 2) for exc in act_cat_mixing.exchanges()]
unit = [bw.get_activity(exc['input'])['unit'] for exc in act_cat_mixing.exchanges()]
df_cath_mixing = pd.DataFrame()
df_cath_mixing[['Input product', 'Activity formula', 'Default amount', 'Unit']] = np.array([input, formula, amount, unit]).T
pd.options.display.max_colwidth = 100
df_cath_mixing

,Input product,Activity formula,Default amount,Unit
0,cathode slurry,cathode_slurry,118.38,kilogram
1,binder solvent (NMP),(binder_solvent_nmp/py_am_mixing_total)-binder_solvent_recovered,1.65,kilogram
2,cathode active material (LFP),cathode_active_material_lfp/py_am_mixing_total,0.0,kilogram
3,cathode active material (LMO),cathode_active_material_lmo/py_am_mixing_total,0.0,kilogram
4,cathode active material (NCA),cathode_active_material_nca/py_am_mixing_total,0.0,kilogram
5,cathode active material (NMC333),cathode_active_material_nmc333/py_am_mixing_total,0.0,kilogram
6,cathode active material (NMC532),cathode_active_material_nmc532/py_am_mixing_total,0.0,kilogram
7,cathode active material (NMC622),cathode_active_material_nmc622/py_am_mixing_total,0.0,kilogram
8,cathode active material (NMC811),cathode_active_material_nmc811/py_am_mixing_total,0.0,kilogram
9,cathode binder (PVDF),cathode_binder_pvdf/py_am_mixing_total,4.6,kilogram


## Update Brightway project parameters
The battery design outputs are used as change the project parameters in Brightway and update the activities amounts accordingly

In [10]:
#Change BatPaC output to Brightway project names (non-alphanumeric keys)
battery_design_param = bat_lca.output_as_bw_param(result_dict)


#Update the project parameters and recalculate activity amounts in Brightway:
bat_lca.update_param_battery_bw(battery_design_param)



In [17]:
battery_design_param

{'anode_active_material_sio': 0.0,
 'anode_active_material_natural_graphite': 0,
 'anode_active_material_synthetic_graphite': 72.1337407906349,
 'anode_binder_cmc': 0.8832702953954661,
 'anode_binder_additive_sbr': 0.5888468635969775,
 'anode_carbon_black': 0.0,
 'anode_current_collector_cu_10um': 45.922648656479105,
 'anode_current_collector_cu_11um': 0,
 'anode_current_collector_cu_12um': 0,
 'anode_current_collector_cu_13um': 0,
 'anode_current_collector_cu_14um': 0,
 'anode_current_collector_cu_6um': 0,
 'anode_current_collector_cu_7um': 0,
 'anode_current_collector_cu_8um': 0,
 'anode_current_collector_cu_9um': 0,
 'battery_jacket': 86.00444276934516,
 'battery_jacket_al': 32.313502509079385,
 'battery_jacket_fe': 51.98983942414179,
 'battery_jacket_insulation': 1.7011008361239872,
 'battery_management_system': 3.5776000000000003,
 'battery_pack': 628.2019987392417,
 'busbar': 0.0,
 'cathode_active_material_50_50_nmc532_lmo': 0,
 'cathode_active_material_lfp': 0,
 'cathode_active_

In [11]:
amount_updated = [round(exc['amount'], 2) for exc in act_cat_mixing.exchanges()]

if 'Updated amount' not in df_cath_mixing.columns:
    df_cath_mixing.insert(3, 'Updated amount', 0) 

df_cath_mixing['Updated amount'] = pd.DataFrame(amount_updated)


df_cath_mixing

,Input product,Activity formula,Default amount,Updated amount,Unit
0,cathode slurry,cathode_slurry,118.38,268.08,kilogram
1,binder solvent (NMP),(binder_solvent_nmp/py_am_mixing_total)-binder_solvent_recovered,1.65,1.31,kilogram
2,cathode active material (LFP),cathode_active_material_lfp/py_am_mixing_total,0.0,0.00,kilogram
3,cathode active material (LMO),cathode_active_material_lmo/py_am_mixing_total,0.0,0.00,kilogram
4,cathode active material (NCA),cathode_active_material_nca/py_am_mixing_total,0.0,0.00,kilogram
5,cathode active material (NMC333),cathode_active_material_nmc333/py_am_mixing_total,0.0,0.00,kilogram
6,cathode active material (NMC532),cathode_active_material_nmc532/py_am_mixing_total,0.0,0.00,kilogram
7,cathode active material (NMC622),cathode_active_material_nmc622/py_am_mixing_total,0.0,0.00,kilogram
8,cathode active material (NMC811),cathode_active_material_nmc811/py_am_mixing_total,0.0,0.00,kilogram
9,cathode binder (PVDF),cathode_binder_pvdf/py_am_mixing_total,4.6,3.66,kilogram


## Calculate environmental impacts in Brightway

In [21]:
lcia_results = {}

#Calculate emissions for all ReCiPe midpoint V.1.13
methods = [method for method in bw.methods if 'ReCiPe Midpoint (H) V1.13 no LT' in str(method)]

# Set up LCI in Brightway with total pack weight as function unit:
battery_production =bw.Database('battery_production').get(('025dcd4c91fc4eeda276f5fff9ad198e_copy5'))
fu = {battery_production: battery_design_param['battery_pack']}
pack_lca = bw.LCA(fu, methods[0])
pack_lca.lci()
pack_lca.lcia()

# Calculate scores:
for impact_category in methods:
    pack_lca.switch_method(impact_category)
    pack_lca.lcia()
    lcia_results [impact_category] = {}
    lcia_results [impact_category]['score'] = pack_lca.score
    lcia_results [impact_category]['unit'] = bw.Method(impact_category).metadata['unit']

In [22]:
df_total = pd.DataFrame.from_dict(lcia_results).T
df_impact_single = pd.DataFrame()
df_impact_single['score pack'] = df_total['score'].astype(float).round(2)
df_impact_single['score kWh'] = (df_total['score']/battery_design_param['battery_capacity']).astype(float).round(4)
df_impact_single['score kg'] = (df_total['score'].astype(float).round(2)/battery_design_param['battery_pack']).astype(float).round(4)
df_impact_single['unit'] = df_total['unit'] 
df_impact_single


score pack  \
ReCiPe Midpoint (H) V1.13 no LT metal depletion                 MDP        14801.03   
                                agricultural land occupation    ALOP         282.04   
                                climate change                  GWP100      5444.59   
                                fossil depletion                FDP         1839.29   
                                freshwater ecotoxicity          FETPinf       13.52   
                                freshwater eutrophication       FEP            0.32   
                                human toxicity                  HTPinf      1504.47   
                                ionising radiation              IRP_HE       549.61   
                                marine ecotoxicity              METPinf       19.98   
                                marine eutrophication           MEP            2.52   
                                natural land transformation     NLTP          -0.81   
                                ozone depletion                 ODPinf         0.00   
                                particulate matter formation    PMFP          15.25   
                                photochemical oxidant formation POFP          21.38   
                                terrestrial acidification       TAP100        40.15   
                                terrestrial ecotoxicity         TETPinf        1.42   
                                urban land occupation           ULOP          77.63   
                                water depletion                 WDP          306.82   

                                                                         score kWh  \
ReCiPe Midpoint (H) V1.13 no LT metal depletion                 MDP       165.2581   
                                agricultural land occupation    ALOP        3.1491   
                                climate change                  GWP100     60.7906   
                                fossil depletion                FDP        20.5363   
                                freshwater ecotoxicity          FETPinf     0.1509   
                                freshwater eutrophication       FEP         0.0036   
                                human toxicity                  HTPinf     16.7979   
                                ionising radiation              IRP_HE      6.1366   
                                marine ecotoxicity              METPinf     0.2231   
                                marine eutrophication           MEP         0.0281   
                                natural land transformation     NLTP       -0.0090   
                                ozone depletion                 ODPinf      0.0000   
                                particulate matter formation    PMFP        0.1703   
                                photochemical oxidant formation POFP        0.2387   
                                terrestrial acidification       TAP100      0.4483   
                                terrestrial ecotoxicity         TETPinf     0.0158   
                                urban land occupation           ULOP        0.8668   
                                water depletion                 WDP         3.4257   

                                                                         score kg  \
ReCiPe Midpoint (H) V1.13 no LT metal depletion                 MDP       25.2909   
                                agricultural land occupation    ALOP       0.4819   
                                climate change                  GWP100     9.3033   
                                fossil depletion                FDP        3.1428   
                                freshwater ecotoxicity          FETPinf    0.0231   
                                freshwater eutrophication       FEP        0.0005   
                                human toxicity                  HTPinf     2.5707   
                                ionising radiation              IRP_HE     0.9391   
                   

#  LCA cradle-to-gate of several battery systems: size effect (Example 2)
To understand the impact of battery size on pack emissions, in the following example several designs with varying pack energy (30)

The following example calculates the emissions of different battery pack sizes ranging from 30 to 130 kWh (5 kWh step)

## Solve several battery systems in BatPaC

In [3]:
dict_design = {}

for energy in range(30, 130+1, 10):
    battery = bd.Battery_system ( vehicle_type='EV', 
                                electrode_pair='NMC622-G (Energy)', 
                                cells_per_module=24, 
                                modules_per_row=6,
                                rows_of_modules=2, 
                                cells_in_parallel=1,
                                silicon_anode= 0.00, 
                                pack_energy=energy, 
                                calculate_fast_charge = 'Yes',
                                max_charging_time = 33,  
                                max_charge_power = 135, 
                                available_energy=94)
    #Establish parameter dictionary
    parameter_dict = battery.parameter_dictionary()

    #Save each BatPaC output in design dictionary:
    dict_design[energy] = parameter_dict
    
#Solve in BatPaC. Make sure BatPaC is closed!
result_dict_all = bd.solve_batpac_battery_system_multiple (
    batpac_path = path_batpac,
    parameter_dict_all=dict_design, 
    visible=False)

100%|██████████| 11/11 [00:48<00:00,  4.39s/it]

esult_all.picklettery designs. Saved results as c:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\example notebooks


In [4]:
df_mc_all = pd.DataFrame.from_dict([result_dict_all[x]['material_content_pack'] for x in result_dict_all.keys()])
df_mc_all['capacity'] = [result_dict_all[x]['general_battery_parameters']['pack_energy_kWh'] for x in result_dict_all.keys()]
df_mc_all = df_mc_all[df_mc_all>0].dropna(axis=1).set_index('capacity')
df_mc_all.T

capacity,30.0,40.0,50.0,60.0,70.0,80.0,90.0,100.0,110.0,120.0,130.0
anode active material (synthetic graphite),26.389262,34.921758,43.429169,51.919316,60.396842,68.864771,77.325203,85.779665,94.220140,102.664125,111.104751
anode binder (CMC),0.323134,0.427613,0.531786,0.635747,0.739553,0.843242,0.946839,1.050363,1.153716,1.257112,1.360466
anode binder additive (SBR),0.215423,0.285076,0.354524,0.423831,0.493035,0.562161,0.631226,0.700242,0.769144,0.838074,0.906978
anode current collector Cu (10um),10.360331,13.448738,16.502425,19.531641,22.542469,25.538875,28.523612,31.498685,34.462389,37.421730,40.374978
battery jacket,59.883673,63.989311,67.615269,70.900557,73.927748,76.750640,79.406600,81.922832,84.317328,86.610772,88.813648
battery jacket Al,20.512764,22.346814,23.970801,25.445603,26.807381,28.079697,29.278915,30.416949,31.501638,32.542112,33.542931
battery jacket Fe,38.303852,40.476513,42.391003,44.122135,45.714350,47.196604,48.589006,49.906198,51.157903,52.355181,53.503707
battery jacket insulation,1.067057,1.165983,1.253465,1.332819,1.406016,1.474339,1.538679,1.599686,1.657787,1.713479,1.767010
battery management system,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600,3.577600
battery pack,242.683480,288.559184,332.907133,376.172235,418.618765,460.418897,501.692217,542.525786,584.180056,624.308680,664.154280


## Update parameters and solve LCA in Brightway
The life cycle emissions for each design are calculated in Brightway for each battery design. This is relatively time consuming as the project parameters are updated for each battery design. A faster approach is presented in example 3 below. 


In [5]:
lcia_results_all = {}

methods = [method for method in bw.methods if 'ReCiPe Midpoint (H) V1.13 no LT' in str(method)]

#Calculate emissions for GWP100 ReCiPe midpoint V.1.13
cc = [method for method in bw.methods if 'ReCiPe Midpoint (H) V1.13 no LT' in str(method) and 'climate change' in str(method)]

#Update Brightway parameters for each battery system:
for battery_design in result_dict_all.keys():
    result_dict_all[battery_design]['brightway_parameters'] = bat_lca.output_as_bw_param(result_dict_all[battery_design])
    battery = result_dict_all[battery_design]
    bat_lca.update_param_battery_bw(battery['brightway_parameters'])

    battery_production =bw.Database('battery_production').get(('025dcd4c91fc4eeda276f5fff9ad198e_copy5'))
    fu = {battery_production: battery['brightway_parameters']['battery_pack']}
    battery_lca = bw.LCA(fu, methods[2])
    battery_lca.lci()
    battery_lca.lcia()
    lcia_results_all [battery_design] = {}
    lcia_results_all [battery_design]['score'] = battery_lca.score
    lcia_results_all [battery_design]['unit'] = bw.Method(cc[0]).metadata['unit']
    lcia_results_all [battery_design]['capacity-kwh'] =battery['general_battery_parameters']['pack_energy_kWh']
    


In [ ]:
result_total = pd.DataFrame.from_dict(lcia_results_all)
emissions = list(result_total.loc['score']/result_total.loc['capacity-kwh'])


## Results
The non-linear relation between the impact score and battery pack size is presented below (top figure). The non-linearity is caused by the increase in energy density (bottom figure)

In [6]:
result_total = pd.DataFrame.from_dict(lcia_results_all)
emissions = list(result_total.loc['score']/result_total.loc['capacity-kwh'])

fig, ax = plt.subplots(2,1, sharex=True, figsize=(6,6))
ax[0].scatter(result_total.loc['capacity-kwh'], emissions,color='black')
ax[0].plot( result_total.loc['capacity-kwh'],emissions,color='r')
ax[1].scatter(result_total.keys(),[result_dict_all[x]['general_battery_parameters']['pack_energy_kWh']/result_dict_all[x]['general_battery_parameters']['battery_system_weight']*1000 for x in result_dict_all.keys()], c='black')
ax[1].plot(result_total.keys(),[result_dict_all[x]['general_battery_parameters']['pack_energy_kWh']/result_dict_all[x]['general_battery_parameters']['battery_system_weight']*1000 for x in result_dict_all.keys()], c='red')

ax[0].set_ylabel('kg CO2-eq')
ax[1].set_ylabel(r'wh kg$^{-1}$')

ax[0].set_title('GWP per kWh battery', fontsize=14)
ax[1].set_title('Gravimetric energy density', fontsize=14)
for x in ax.flatten():
    x.grid(True)

fig.tight_layout()

Error: Canceled future for execute_request message before replies were done

# Modular LCA single battery (Example 3)
The iterative calculation of multiple battery designs by changing the parameters in Brightway is relatively slow to solve many battery designs. 

<br>Instead, a more efficient way is the use of precalculated gate-to-gate LCA modules based on https://link.springer.com/article/10.1007/s11367-015-1015-3. 

## Establish base product-module matrix ($A'$)



The first step is to establish the base product-module matrix ($A'$). 
To obtain this matrix, we first establish a new BW database containing all relevant modules which represent our foreground system<br>




<!-- set to 1 kg and all battery-production processes set to 1 kg of battery (e.g. anode coating required for 1 kg of battery). 
 -->

In [17]:
df_cut_off_m = pd.read_excel(r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\batt_sust_model\data\process_formulas.xlsx", sheet_name='cut_off_modules').set_index('process')

In [18]:
# %%capture

activities= []
count = []
#check if db already established:

if 'cut_off' in bw.databases:
    cut_off_db =  bw.Database('cut_off')
if 'cut_off' not in bw.databases:
    cut_off_db =  bw.Database('cut_off')
    cut_off_db.register()

def format_me(list_of_activities, database_obj):
    return {(database_obj.name, obj['code']): obj for obj in list_of_activities}



for row in df_cut_off_m.iterrows():
    act = bw.Database(row[1]['database']).get(row[1]['code'])
    if act not in bw.Database('cut_off'):
        act.copy(database="cut_off")
        act.save()
    
bat_lca.add_activity_parameters('cut_off')


# RELINK TO CUT-OFF DATABASE.. NOW USE AB BUT USING THIS FUNCTION:
relink_exchanges_existing_db

In [38]:
#First reset all BW parameters for the base battery design
bat_lca.update_param_battery_bw(battery_design_param)

In [24]:
# Cut off the exchanges if activity if present in the 'cut off database'. 
# E.g. 6um copper foil production is present in the 'cut of database' and will be cut in the anode coating process. All cut-off exchanges are set to zero

a = bat_lca.modules_with_cuts('cut_off')

{'battery jacket aluminium': {'key': ('cut_off',
   '0347b14c970a4bffb6ebeb554a6f292f'),
  'output': 'battery jacket aluminium',
  'amount': 1,
  'cuts': {('cut_off',
    'd17001e437b44b75aa46e310c2c5d3fd'): ['aluminium, wrought alloy', -1]}},
 'cathode current collector Al (15um) production': {'key': ('cut_off',
   '33ca06e0e25c4fb5b0e6683ea611fd19'),
  'output': 'cathode current collector Al (15um)',
  'amount': 1,
  'cuts': {('cut_off',
    '2840b563fe664ca497d14cfe2dad96c9'): ['transport, freight train', -0.0438],
   ('cut_off',
    'd17001e437b44b75aa46e310c2c5d3fd'): ['aluminium, wrought alloy', -1]}},
 'cathode active material (NMC532) import': {'key': ('cut_off',
   '3fff44585b4a466188032ee738b84249'),
  'output': 'cathode active material (NMC532)',
  'amount': 1,
  'cuts': {('cut_off',
    '64b0358e1bd049799e245fc0ffdf7811'): ['lithium carbonate', -0.381],
   ('cut_off', '1b3aa7500d9f45a19c85ef72742385f4'): ['NMC532 precursor',
    -0.945]}},
 'cathode slitting': {'key': ('cut

In [28]:
# Divide all battery production activities by total battery system weight:
bat_product_act = [act['name'] for act in bw.Database('battery_production')]
for act in bat_product_act:
    pack_weight = battery_design_param['battery_pack']
    a[act]['amount'] = a[act]['amount']/pack_weight
    for exc in a[act]['cuts'].keys():
        a[act]['cuts'][exc][1] = a[act]['cuts'][exc][1]/pack_weight
        
#Cut modules:
bat_lca.cut_modules_to_zero(a)

# Establish product-module dataframe of base system
A_prime=bat_lca.modular_technology_matrix(a)

A_prime

,"market group for heat, district or industrial, natural gas",electrolyte filling and sealing,electrolyte (LMO) production,module interconnects production,electrolyte (LFP) production,cathode active material (NCA) import,"market for transport, freight, sea, container ship",market for module elastomer pads,battery management system production,cathode coating and drying,...,spacer for gas release production,Cobalt sulfate production,waste anode slurry handling,cathode binder (PVDF) production,cathode electrode scrap handling,"sheet rolling, steel",cell container production,cathode active material (LMO) import,NCA precursor production,cathode binder solvent waste recovery
"heat, district or industrial, natural gas for battery production",1,-2.559744,0,0.0,0,0.00,0,0.0,0.0,-2.166743,...,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000
unformated cell,0,0.564330,0,0.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000
electrolyte (LMO),0,0.000000,1,0.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000
module interconnects,0,0.000000,0,1.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000
electrolyte (LFP),0,0.000000,0,0.0,1,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"sheet rolling, steel",0,0.000000,0,0.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,1.0,0.0,0.0,0.0,0.000000
cell container,0,0.000000,0,0.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,1.0,0.0,0.0,0.000000
cathode active material (LMO),0,0.000000,0,0.0,0,0.00,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.000000
NCA precursor,0,0.000000,0,0.0,0,-0.95,0,0.0,0.0,0.000000,...,0.0,0.0,0,0,0,0.0,0.0,0.0,1.0,0.000000


In [42]:
A_prime.to_excel(r'C:\Users\Joris\Downloads\output.xlsx')

In [129]:
module_dict = {}

for db in bw.databases:
    if db in ['biosphere3','cut_off_production', 'cut_off_materials', 'uncertainty','manufacturing_waste_scrap']:
        continue
    activities = [act['name'] for act in bw.Database(db)]
    for process in A_prime.columns:
        
        if process in activities:
            code = [act['code'] for act in bw.Database(db) if act['name'] == process][0]
            location = [act['location'] for act in bw.Database(db) if act['name'] == process]

            module_dict [process] = [db, code, location ]
        

{'material_content_pack': {'anode active material (SiO)': 0.0,
  'anode active material (natural graphite)': 0,
  'anode active material (synthetic graphite)': 75.56726444713466,
  'anode binder (CMC)': 0.9253134422098122,
  'anode binder additive (SBR)': 0.6168756281398748,
  'anode carbon black': 0.0,
  'anode current collector Cu (10um)': 27.904097517445724,
  'anode current collector Cu (11um)': 0,
  'anode current collector Cu (12um)': 0,
  'anode current collector Cu (13um)': 0,
  'anode current collector Cu (14um)': 0,
  'anode current collector Cu (6um)': 0,
  'anode current collector Cu (7um)': 0,
  'anode current collector Cu (8um)': 0,
  'anode current collector Cu (9um)': 0,
  'battery jacket': 78.8668549917414,
  'battery jacket Al': 29.035042222542863,
  'battery jacket Fe': 48.30621346149433,
  'battery jacket insulation': 1.525599307704213,
  'battery management system': 3.5776000000000003,
  'battery pack': 493.15240235463983,
  'busbar': 0.0,
  'cathode active materia

## Establish environmental impact matrix of modules ($H_m$)
Precalculation of each cut-off module

In [34]:
# list_fu = [
#         {bw.get_activity(a[act]["key"])['database']: a[act]["amount"]}
#         for act in a.keys()
#     ]

In [33]:
# bw.calculation_setups["multi_lca"] = {"inv": list_fu, "ia": impacts}


In [32]:
# MultiLCA = bw.MultiLCA("multi_lca")

In [25]:
#Select all Recipe Midpoint impacts:
impacts = [m for m in bw.methods if 'ReCiPe Midpoint (H) V1.13' in str(m) and 'no LT' in str(m)]
H_m = bat_lca.lcia_modules (a, impacts)

Multiplication of the modular scaling factor ($s'$) with the modular emission matrix ($H_m$) to get the total emisions of the system ($h$). <br>
Scaling vector is calculated with the general LCA formula:<br>
$$s' = A'^{-1}y'$$
where $y'$ is the final demand vector.

In [26]:
df_H_modular = pd.DataFrame(H_m[:,:].T,index=impacts, columns=A_prime.columns)

NameError: name 'A_prime' is not defined

In [65]:
df_H_modular.to_excel(r"C:\Users\Joris\Downloads\output_h.xlsx")

In [29]:
#Calculate for design 1:
pack_design =battery_design_param
#Parameter shortcuts:
pack_weight = battery_design_param['battery_pack']
capacity = battery_design_param['battery_capacity']

#Inverse the A' matrix:
A_inv = pd.DataFrame(np.linalg.pinv(A_prime.values), A_prime.columns, A_prime.index)

#Establish final product demand vector for 1 battery based on pack weight
y_prime = pd.Series(data = 0, index=A_prime.index)
y_prime.loc['battery pack'] = pack_weight

#Calculate scaling vector:
s_prime = A_inv.dot(y_prime)


Error: Canceled future for execute_request message before replies were done

## Calculate total emissions of the system ($h$)

Multiplication of the scaling vector with the pre-calculated emissions of the modules:

$$h=s'H_m$$

In [ ]:
# Emissions for all categories and modules:
h_all =  np.multiply(np.transpose([s_prime]),H_m)

Top 10 LCA modules with highest CO2-eq emissions:

In [50]:
df_h=pd.DataFrame(h_all[:,2],A_prime.columns, columns=[impacts[2]])
display(df_h.astype(int).sort_values(by=impacts[2], ascending=False).head(10)/capacity)

,"(ReCiPe Midpoint (H) V1.13 no LT, climate change, GWP100)"
cathode active material (NMC622) import,11.976588
market for cobalt hydroxide,9.895187
"market for aluminium, wrought alloy",9.042154
market for nickel sulfate,6.938005
"market group for electricity battery production, medium voltage",5.231938
anode active material (synthetic graphite) import,5.186443
"market for copper, cathode",3.025425
NMC622 precursor production,2.729707
Cobalt sulfate production,2.468110
electrolyte (NMC/NCA) production,2.070028


Total GWP value:

In [51]:
df_h.sum()

(ReCiPe Midpoint (H) V1.13 no LT, climate change, GWP100)    6224.329575
dtype: float64

In [53]:
df_h.to_excel(r"C:\Users\Joris\Downloads\output_h.xlsx")

In [ ]:
modules_dict =a

for module in modules_dict.keys():
    if modules_dict[module]["cuts"]:
        act = bw.get_activity(modules_dict[module]["key"])
        cut_exchanges = [
            exc
            for exc in act.exchanges()
            if exc["input"] in modules_dict[module]["cuts"].keys()
        ]
        for cut_exc in cut_exchanges:
            try:  # Exchanges with formulas
                if cut_exc["formula"] is not None:
                    cut_exc["amount"] = 0  # Amount to zero for cut-off modules
                    cut_exc.save()
            except KeyError:  # If exchange has no formula, set formula to amount
                print (cut_exc)
                # make new activity parameter:
                cut_exc["formula"] = cut_exc["amount"]
                cut_exc.save()
                bw.parameters.add_exchanges_to_group("my group", act)
                cut_exc["amount"] = 0
                cut_exc.save()


pd.DataFrame(module_dict.values(), index= module_dict.keys()).to_clipboard()

Visualise emissions by module:

In [38]:
#Import process and product category mapping:
category_mapping=pd.read_excel(r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\process_product_type_contribution.xlsx", sheet_name='values').fillna(0)


def plot_module_impact (df_lcia_modules, df_category_mapping, title=None):
    """ Plot the impact of modules per module category"""
    df = df_category_mapping
    df.set_index('process', inplace=True)
    df=df.join(df_lcia_modules).groupby(['component category']).sum()
    category = df_lcia_modules.columns[0]
#     df.insert(loc=0, column='battery_system', value=lcia_modules)   
    df=df.sort_values(by=category, ascending=False)
    axes = df.T.plot.bar(stacked=True, legend=True, fontsize=14, figsize=(12,8), zorder=3)
    axes.grid(zorder=0, axis='y')
    axes.set_title(f'{title} \n CO2-eq emissions per kWh', fontsize=18)
    axes.set_ylabel('kg CO2-eq', fontsize = 14)
    axes.legend(loc='center left',bbox_to_anchor=(1.0, 0.5), fontsize=14)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()



f = plot_module_impact(df_h/capacity,category_mapping, title=f'{round(capacity)} kWh')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Joris\\OneDrive - Newcastle University\\PhD\\Models\\ME optimisation model\\ME_battery_supply_chain\\Model\\data\\process_product_type_contribution.xlsx'

## Modular LCA for multiple batteries


In [ ]:
battery = bd.Battery_system ( vehicle_type='EV', electrode_pair='NMC622-G', cells_per_module=24, modules_per_row=6,rows_of_modules=2, 
                             cells_in_parallel=1,silicon_anode= 0.00, pack_energy=82, available_energy=94).parameter_dictionary()
battery_designs_params=battery

In [ ]:
dict_result = bd.solve_batpac_battery_system (batpac_path = path_batpac, parameter_dict=battery, visible=False)

In [ ]:
# chemistry range
cath      = ['NMC333-G', 'NMC532-G','NMC622-G', 'NMC811-G','NCA-G','LFP-G','NMC532/50%/LMO  - G', 'LMO-G']
battery_designs_params = {}

# Establish a dictionary with a class of all battery designs:
for x in cath:
    name = f'{x} 23.5'
    battery = bd.Battery_system ( vehicle_type='EV', electrode_pair=x, cells_per_module=24, modules_per_row=6,rows_of_modules=2, 
                             cells_in_parallel=1,silicon_anode= 0.00, pack_energy=82, available_energy=94).parameter_dictionary()
    battery_designs_params[name]=battery

# Solve all battery systems in BatPaC.
result_dict_all = bd.solve_batpac_battery_system_multiple (batpac_path = path_batpac, parameter_dict_all=battery_designs_params, visible=False)

from tabulate import tabulate
headers = result_dict_all.keys()
data = [
        ['kWh', *[result_dict_all[pack]['general_battery_parameters']['pack_energy_kWh'] for pack in result_dict_all.keys()]]
       ]

for battery_design in result_dict_all.keys():
    battery = result_dict_all[battery_design]
    result_dict_all[battery_design]['brightway_parameters'] = bat_lca.output_as_bw_param(result_dict_all[battery_design])

print(tabulate(data, headers=headers))


In [ ]:
path_comp_type_linkage = r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\component_type_linkage.xlsx"

df_types = pd.read_excel(path_comp_type_linkage, index_col='component')

In [ ]:
result = result_dict_all['NMC333-G 23.5']['material_content_pack']

df_types['result'] = df_types.index.map(result).fillna(0)
df_types = df_types[(df_types != 0).all(1)]
df_types = df_types.groupby(['component_type', 'part_off']).sum()
df_types.sort_values(by='part_off', ascending=True, inplace=True)
df_types = df_types.reset_index(level=[1])

In [ ]:
df = df_types.drop('part_off', axis=1).sort_values(by='result', ascending=False)
df.T.plot.bar(stacked=True)

## Update parameter and activity formulas 

The activity and parameter formulas need to be calculcated for each battery design system. To improve the calculation speed, this is done outside of Brightway using the functions below. The Brightway formula strings are stored in a separate 

In [ ]:
# 1. Calculate all formulas in the project parameters
import re

#Import all activity formulas:
func_df = pd.read_excel(r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\bw_activity_functions.xlsx", sheet_name = 'activity_functions', index_col=[1,2])
func_dict = func_df.T.to_dict()

# Calculate and append parameters to dictionary (project parameters in Brightway):

def project_parameters_brightway ():
    """ Returns dictionary with amount and formula of the Brightway project parameter"""
    project_param_dict = {}
    for param in ProjectParameter:
        if param.formula is None:
            param.formula=0
        project_param_dict[param.name] = {'amount':param.amount, 'formula':param.formula}
    return project_param_dict

def update_project_parameter_amount (project_param_dict,design_dict):
    """ Update the 'amount' in the Brightway project parameter dictionary based on battery design"""        
    for param in design_dict['design_parameters'].keys():
        project_param_dict[param]['amount'] = design_dict['design_parameters'][param]
    return project_param_dict


def update_project_parameter_formulas (project_parameters_formulas):
    """ Calculates the project parameter formulas recursively.
    
    Args:
        project_parameters_dict (dict): dictionary with Brightway project parameters
        
    Return:
        dictionary with parameter name (key) and amount (value)
    """
    param_dict_2= {a:b['amount'] for a, b in project_parameters_formulas.items() if b['formula'] == 0}
    for param in project_parameters_formulas.keys():
        if project_parameters_formulas[param]['formula'] != 0:
            
            def calc_amount_formula(param,param_dict_2):
                """ Recursive function, calls itself if parameter name in project formula is based on a different formula"""
                try:
                    amount=eval(project_parameters_formulas[param]['formula'], param_dict_2)
                    return amount
                except NameError as Argument:
                    param_error = re.split("['']", str(Argument))[1]
                    amount = calc_amount_formula(param_error,param_dict_2 )
                    param_dict_2[param_error] = amount
                    amount = calc_amount_formula (param, param_dict_2)
                    return amount
            amount = calc_amount_formula(param,param_dict_2)    
            project_parameters_formulas[param]['amount'] = amount
            param_dict_2[param] = amount
    param_dict_amount = {a:b['amount'] for a, b in project_parameters_formulas.items()}
    return param_dict_amount


def get_project_parameters_dict (design_dict):
    """ Returns a dictionary of the Brightway project parameters based on design parameter amounts
    and calculated project formulas
    
    Args:
        design_dict [dict]: design parameters
        
    Return:
        dict: parameter name and amount    
    """
    project_param_dict = project_parameters_brightway()
    #update 'amount' in project parameter based on design dictionary:
    project_param_dict_update = update_project_parameter_amount(project_param_dict, design_dict)        
    #Update project parameters with formulas:
    parameter_dictionary =  update_project_parameter_formulas(project_param_dict_update)  
    
    return parameter_dictionary



def update_module_formulas (design_dict, activity_functions):
    """ Updates parameterised modules based on activity formalas and project parameters 
    
    Args:
        project_parameters (dict): Brightway project parameter values 
        activity_functions (dict): Brightway activity fucntions
        
    Returns:
        Dictionary with updated activity amounts
    """
    output_dict = {}

    parameter_dictionary = get_project_parameters_dict (design_dict)
    
    # Upate all activity formulas based on project parameters:
    for key in activity_functions.keys():
        
        if activity_functions[key]['material_group'] == 'reference product': 
            #Output/reference product is positive
            amount = eval(activity_functions[key]['formula'], parameter_dictionary)   
        else: 
            #Input is negative
            amount = -eval(activity_functions[key]['formula'], parameter_dictionary)
        output_dict[key] = amount 
    return output_dict

  

def update_parameterised_modules (technology_matrix_base, battery_design_dict, activity_functions_dict):
    """ Updates modules based on functions with design specific parameters
    
    Args:
        technology_matrix_base (df): base product-module matrix (A_prime)
        battery_design_dict (dict): dictionary of activity project parameters        
        activity_functions (dict): dictionary of parameterised modules (based on functions) including 
    
    Return:
        df: updated square product-module matrix
    """

    updated_act = update_module_formulas(battery_design_dict, activity_functions_dict)
    battery_weight = battery_design_dict['brightway_parameters']['battery_pack']
    A_matrix = technology_matrix_base.copy(deep=True) # To make sure the default A dataframe is not modified
    bat_product_act = [act['name'] for act in bw.Database('battery_production')]

    for idx in A_matrix.index:
        for col in A_matrix.columns:
            if (col, idx) in updated_act.keys() and col in bat_product_act:
                A_matrix.loc[idx, col] = updated_act[(col, idx)]/battery_weight
            elif  (col, idx) in updated_act.keys():
                A_matrix.loc[idx, col] = updated_act[(col, idx)]    
    return A_matrix





## Calculate emissions and visualise results

In [ ]:
total_emission_kwh = []
module_emissions = {}

for key in result_dict_all.keys():
    
    capacity = result_dict_all[key]['brightway_parameters']['battery_capacity']
    pack_weight = result_dict_all[key]['brightway_parameters']['battery_pack']
    A_prime_temp = update_parameterised_modules(A_prime,result_dict_all[key], func_dict)
    
    #Inverse the A' matrix:
    A_inv = pd.DataFrame(np.linalg.pinv(A_prime_temp.values), A_prime_temp.columns, A_prime_temp.index)

    #Establish final product demand vector for 1 battery based on pack weight
    y_prime = pd.Series(data = 0, index=A_prime_temp.index)
    y_prime.loc['battery pack'] = pack_weight

    #Calculate scaling vector:
    s_prime = A_inv.dot(y_prime)
    # GWP100:
    emission = H_m[:,2]*s_prime
    module_emissions[key]=emission

    total_emission_kwh.append(emission.sum()/capacity)
    
#Plot total emissions
plt.bar(result_dict_all.keys(), total_emission_kwh)
plt.xticks( rotation='vertical')



In [ ]:
category_mapping=pd.read_excel(r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\process_product_type_contribution.xlsx", sheet_name='values').fillna(0)
category_mapping.set_index('process', inplace=True)

       
df = category_mapping.groupby(['aggregated']).sum()


for key in module_emissions.keys():

        df_module_emission = pd.DataFrame(module_emissions[key])
        df_temp =category_mapping.join(df_module_emission).groupby(['aggregated']).sum()
        df[key]=df_temp

        

ax=(df/capacity).sort_values(by=list(result_dict_all.keys())[0], ascending=False).T.plot.bar(stacked=True)
ax.legend(bbox_to_anchor=(1.1, 1.05))
ax.set_ylabel('GWP100 impact [kg CO2-eq]')


In [ ]:
openpyxl.load_workbook(r'3_MC_battery_pack_material.xlsx')

# calculate battery design

In [ ]:
import itertools
from tqdm.notebook import tqdm

cath      = ['NMC333-G', 'NMC532-G','NMC622-G', 'NMC811-G','NCA-G','LFP-G','NMC532/50%/LMO  - G']
ano       = ['synthetic', 'natural']
sil       = [0.0,0.08]
cell_mod  = [12,34]  # Based on min and max values in BatPaC
energy    =  [75]
battery_designs_params = {}
count = 0
for cat, an, si, cell, ene in tqdm(itertools.product(cath, ano, sil, cell_mod, energy)):
#     print (count, cat, an, si, cell, ene)

    
    battery = bd.Battery_system ( vehicle_type='EV', electrode_pair=cat, 
                                            silicon_anode= si, pack_energy = ene, graphite_type=an,cells_per_module=cell ).parameter_dictionary()
    count +=1

    battery_designs_params[count]=battery

# result_dict_all = bd.solve_batpac_battery_system_multiple (batpac_path = path_batpac, parameter_dict_all=battery_designs_params, visible=False)


In [ ]:
result_dict_all = bd.solve_batpac_battery_system_multiple (batpac_path = path_batpac, parameter_dict_all=battery_designs_params, visible=True)


In [ ]:
#Establish correpsonding processs emissions matrix H:

df_emission = pd.DataFrame(impact_all, index=A.columns, columns=impacts).T

In [ ]:
def calculate_modular_A (technology_matrix_default, pre_calculated_modules, battery_design_dictionary, name,  production_act, reference_prod):  
    
    updated_act = update_formulas(battery_design_dictionary)
    battery_weight = battery_design_dictionary['brightway_parameters']['battery_pack']
    capacity = battery_design_dictionary['brightway_parameters']['battery_capacity']
    A_matrix = technology_matrix_default.copy(deep=True) # To make sure the default A dataframe is not modified
    h = pre_calculated_modules.copy(deep=True) 
    
    
    # Establish A frame for battery production. Production processes and process outputs (reference products) have unique name, except battery pack
    for idx in A_matrix.index:
        for col in A_matrix.columns:
            if (col, idx) in updated_act.keys() and col in production_act:
                A_matrix.loc[idx, col] = updated_act[(col, idx)]
            elif  (col, idx) in updated_act.keys():
                A_matrix.loc[idx, col] = updated_act[(col, idx)]
                
    #Battery pack in unit, not kg:
    A_matrix.loc['battery pack', 'module and pack assembly']=1
     
    #Unique name for battery production processes and products:
    bat_product_act_unique = [act +f' {name}' for act in production_act]
    reference_prod_unique = [product+f' {name}' if product !='battery pack' else product for product in reference_prod]
    new_act_names = dict(zip(production_act,bat_product_act_unique))
    new_prod_names = dict(zip(reference_prod, reference_prod_unique))
    
    A_matrix.rename(columns=new_act_names, inplace=True)
    A_matrix.rename(index=new_prod_names, inplace = True)
    h.rename(columns=new_act_names, inplace=True)    
    return A_matrix[bat_product_act_unique], h[bat_product_act_unique]

In [ ]:
# Calculate all process parameters for specfici design:
#Get battery production activities and reference products:
production_act = []
reference_prod  = []
for act in bw.Database('battery_production'):
    production_act.append(act['name'])
    for exc in act.exchanges():
        if exc['type'] == 'production':
            reference_prod.append(bw.get_activity(exc['input'])['reference product'])


A_wo_production = A.drop(production_act, axis=1).drop(reference_prod)
h_wo_production = df_emission.drop(production_act, axis=1)

list_df_A = []
list_df_h = []
for x in result_dict_all.keys():
    df_A, df_h = calculate_modular_A(A,df_emission,result_dict_all[x], x, production_act,reference_prod )
    list_df_A.append(df_A)
    list_df_h.append(df_h)
prod_df = pd.concat(list_df_A, axis=1).fillna(0)
h_array = pd.concat(list_df_h, axis=1)
A_non_square = pd.concat([prod_df,A_wo_production], axis=1).fillna(0)
h_non_square = pd.concat([h_array, h_wo_production], axis=1).fillna(0)


In [ ]:
df.to_excel('3_PAR_battery_design_parameters.xlsx')

In [ ]:
import pickle

a = {'hello': 'world'}

# with open(r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Data\Data dump\50_battery.pickle', 'wb') as handle:
#     pickle.dump(result_dict_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Data\Data dump\battery_all.pickle', 'rb') as handle:
    result_dict_all = pickle.load(handle)

weight = []

for x in result_dict_all.keys():
    weight.append(result_dict_all[x]['material_content_pack']['battery pack'])
    
weight.sort()
plt.plot(weight)

In [ ]:
for battery_design in result_dict_all.keys():
    battery = result_dict_all[battery_design]
    result_dict_all[battery_design]['design_parameters'] = bat_lca.output_as_bw_param(result_dict_all[battery_design])

In [ ]:
A_non_square.to_csv(r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Data\Data dump\4_upi_non_square.csv')
h_non_square.to_csv(r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Data\Data dump\4_pe_non_square.csv')

In [ ]:
charging_time = list(range(5,42,1))

bat_dict_all = {}
count = 0
for time in charging_time:
    #Establish battery system class of specific design:
    battery = bd.Battery_system ( vehicle_type='EV', electrode_pair='NMC622-G', cells_per_module=24, modules_per_row=6,rows_of_modules=2, 
                             cells_in_parallel=1,silicon_anode= 0.00, pack_energy=84, available_energy=94, sep_foil_thickness=9, sep_coat_thickness=3, max_charging_time=time)
    bat_dict_all[count] = battery.parameter_dictionary()
    count+=1

    
result_dict_all = bd.solve_batpac_battery_system_multiple (batpac_path = path_batpac, parameter_dict_all=bat_dict_all, visible=False)


In [ ]:
with open(r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Written parts\Figures\Figure data\Appendix_data\charging_time_sensitivity.pickle', 'rb') as handle:
    result_dict_all = pickle.load(handle)
charging_time = list(range(5,42,1))

pack_weight = []
electrode = []

for k,v in result_dict_all.items():
    electrode.append(v['general_battery_parameters']['positive_electride_thickness'])
    pack_weight.append(v['material_content_pack']['battery pack'])


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.scatter(charging_time[0:41-15], pack_weight[0:41-15], label='Battery weight', s=10)
ax1.plot(charging_time[0:41-15], pack_weight[0:41-15], linewidth=1.2)

ax2.scatter(charging_time[0:41-15], electrode[0:41-15], c='r', label='Positive electrode thickness', s=10)
ax2.plot(charging_time[0:41-15], electrode[0:41-15], c='r', linewidth=1.2)

ax1.set_ylabel('Battery weight (kg)')
ax2.set_ylabel ('Electrode thickness (μm)')
ax1.set_xlabel ('Charge time 15-75% SOC (min)')
fig.legend()